# Map shifts in the start and end of rainy seasons

Following some publications for determining start and end of climatological rainfall
* https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016JD025428
* https://journals.ametsoc.org/view/journals/clim/14/22/1520-0442_2001_014_4308_ivotrs_2.0.co_2.xml?tab_body=pdf

Complicated python function here too: https://github.com/rjbombardi/onset_demise_rainy_season

In [ ]:
%matplotlib inline

import os
import sys
import warnings
import numpy as np
import xarray as xr
import pandas as pd
import contextily as ctx
import matplotlib as mpl
import geopandas as gpd
from scipy.signal import find_peaks
from shapely.geometry import box
from odc.geo.geom import Geometry
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _prediction import allNaN_arg
from _utils import round_coords


In [ ]:
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import start_local_dask

client = start_local_dask(mem_safety_margin='2Gb')
client

## Open data

In [ ]:
# ANUClim rain
rain = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/rainfall_ANUClim_daily_5km_1960_2022.nc',
                      chunks=dict(latitude=250,longitude=250, time=-1))['rain']

#CHIRPS daily rainfall
# rain = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/rainfall_CHIRPS_daily_5km_1981_2023.nc',
#                       chunks=dict(latitude=250,longitude=250, time=-1))['rain']

#silo rain
# rain = xr.open_dataset('/scratch/xc0/chad/1km/rain_1km.nc',
#                       chunks=dict(latitude=750,longitude=750, time=-1))['rain']

# rain = rain.resample(time="1W").interpolate("linear").compute()

## Extract climatology differences between epochs for example region

Using Southwest WA as there's been a significant decline in rainfall amounts and timing.

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson')

In [ ]:
# gdf.explore(column='SUB_NAME_7',
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False
#            )

In [ ]:
plotname = 'Carnegie'#'Victorian Alps'  #'Geraldton Hills rainfed crop' 'Weipa Plateau'
# Claraville Plains Eastern Darling Downs rainfed crop Geraldton Hills rainfed crop Highlands-Southern Fall

minx,miny,maxx,maxy = gdf[gdf['SUB_NAME_7']==plotname].geometry.bounds.values[0]
lat=slice(maxy,miny) 
lon=slice(minx,maxx)

rain_subset = rain.sel(longitude=lon, latitude=lat).compute()

In [ ]:
# clim_mean = rain_subset.mean('time')

#first calculate the day-of-year rainfall climatology for the first 20 years
clim_mean_month_early = rain_subset.sel(time=slice('1980','2000')).groupby('time.dayofyear').mean()
#now calculate the long-term mean daily rainfall
clim_mean_early = rain_subset.sel(time=slice('1980','2000')).mean('time')
#calculate the cumulative daily rainfall anomaly
cml_rain_anom_early = (clim_mean_month_early - clim_mean_early).cumsum(dim='dayofyear')

#same again but for the last 20 years of the time series
clim_mean_month_late = rain_subset.sel(time=slice('2001','2023')).groupby('time.dayofyear').mean()
clim_mean_late = rain_subset.sel(time=slice('2001','2023')).mean('time')
cml_rain_anom_late = (clim_mean_month_late - clim_mean_late).cumsum(dim='dayofyear')

# mase to remove zero where there used to be nans
mask = ~np.isnan(clim_mean_month_early.mean('dayofyear'))

### Determine the min and max points of cumulative anomalies

This shows where the SOS and EOS of the rainfall seasons occur.

In [ ]:
eos_doy = cml_rain_anom_early.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().idxmax(dim='dayofyear').values.item()
eos_rain = cml_rain_anom_early.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=eos_doy).values.item()

sos_doy = cml_rain_anom_early.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().idxmin(dim='dayofyear').values.item()
sos_rain = cml_rain_anom_early.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=sos_doy).values.item()

eos_doy_late = cml_rain_anom_late.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().idxmax(dim='dayofyear').values.item()
eos_rain_late = cml_rain_anom_late.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=eos_doy_late).values.item()

sos_doy_late = cml_rain_anom_late.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().idxmin(dim='dayofyear').values.item()
sos_rain_late = cml_rain_anom_late.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=sos_doy_late).values.item()

### Create a plot to demonstrate method

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(8,5))
ax1=ax.twinx()
# clim_mean_month_early.mean(['latitude', 'longitude']).plot()
pl1=(clim_mean_month_early - clim_mean_early).mean(['latitude', 'longitude']).plot(ax=ax,
                                    label='1982-2001 climatological rainfall anomaly', linestyle='--', linewidth=0.75)
pl2=cml_rain_anom_early.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().plot(ax=ax1, label='1982-2001 cumulative rainfall anomaly')

# clim_mean_month_late.mean(['latitude', 'longitude']).plot()
pl3=(clim_mean_month_late - clim_mean_late).mean(['latitude', 'longitude']).plot(ax=ax, 
                                    label='2002-2022 climatological rainfall anomaly',linestyle='--', linewidth=0.75)
pl4=cml_rain_anom_late.mean(['latitude', 'longitude']).rolling(dayofyear=30, min_periods=1).mean().plot(ax=ax1, label='2002-2022 cumulative rainfall anomaly')

#add sos
ax1.plot(sos_doy, sos_rain, 'or')
ax1.annotate('SOS',
            xy=(sos_doy, sos_rain),
            xytext=(10, 15),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

ax1.plot(sos_doy_late, sos_rain_late, 'or')
ax1.annotate('SOS',
            xy=(sos_doy_late, sos_rain_late),
            xytext=(10, 15),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

#add eos
ax1.plot(eos_doy, eos_rain, 'or')
ax1.plot(eos_doy_late, eos_rain_late, 'or')
ax1.annotate('EOS',
            xy=(eos_doy, eos_rain),
            xytext=(-10, -25),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

ax1.annotate('EOS',
            xy=(eos_doy_late, eos_rain_late),
            xytext=(-10, -25),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

plots=pl1+pl2+pl3+pl4
labels = [l.get_label() for l in plots]
ax.legend(plots, labels, loc='upper right')
ax.set_ylabel('Daily rainfall anomaly (mm/day)')
ax1.set_ylabel('Cumulative daily rainfall anomaly (mm)')
ax.axhline(0, c='grey')
ax.grid(alpha=0.75)
ax.set_title(None)
ax1.set_title(plotname)
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/zonal_rainfall_patterns_{plotname}.png',
            bbox_inches='tight', dpi=300);

## Now run australia-wide

### Consider masking very high variance regions

Simpler if we just use annual variance. This has the added advantage of limiting the results to regions where the rainfall gauges are more prominent

In [ ]:
# annual_rain =  rain.sel(time=slice('1960','2022')).resample(time='YS').sum()
# clim_mean = annual_rain.mean('time').compute()
# clim_std = annual_rain.std('time').compute()

# scaled_variance = clim_std / clim_mean

# high_daily_variance = xr.where(scaled_variance < 0.4, 1,0)
# rain = rain.where(high_daily_variance)

# high_daily_variance.plot();
# plt.title(f'% remaining: {(np.sum(high_daily_variance) / np.sum(~np.isnan(clim_mean.where(clim_mean>0))) * 100).item():2.2f}');

In [ ]:
# def stand_anomalies(ds, mean, std):
#     return xr.apply_ufunc(
#         lambda x, m, s: (x - m) / s,
#             ds.groupby("time.dayofyear"),
#             mean,
#             std,
#             dask='allowed'
#     )
    
# ds_clim_mean = rain.groupby("time.dayofyear").mean('time').compute()
# ds_clim_std = rain.groupby("time.dayofyear").std('time').compute()
# rain_anom = stand_anomalies(rain, ds_clim_mean, ds_clim_std).compute()

# del ds_clim_mean
# del ds_clim_std

# anom_mask = xr.where((rain_anom > -3.5) & (rain_anom < 3.5), 1, 0).drop_vars('dayofyear')

# del rain_anom

# rain = rain.compute()
# rain = rain.where(anom_mask).astype('float32')
# rain = rain.chunk(dict(latitude=250,longitude=250, time=-1))

In [ ]:
clim_mean_month_early = rain.sel(time=slice('1980','2000')).groupby('time.dayofyear').mean().compute()
clim_mean_early = rain.sel(time=slice('1980','2000')).mean('time').compute()
cml_rain_anom_early = (clim_mean_month_early - clim_mean_early).cumsum(dim='dayofyear').rolling(dayofyear=30, center=True, min_periods=1).mean()

clim_mean_month_late = rain.sel(time=slice('2001','2022')).groupby('time.dayofyear').mean().compute()
clim_mean_late = rain.sel(time=slice('2001','2022')).mean('time').compute()
cml_rain_anom_late = (clim_mean_month_late - clim_mean_late).cumsum(dim='dayofyear').rolling(dayofyear=30, center=True, min_periods=1).mean()

mask = ~np.isnan(clim_mean_month_early.mean('dayofyear')).compute()

early_clim_max = cml_rain_anom_early.idxmax(dim='dayofyear').where(mask) 
late_clim_max = cml_rain_anom_late.idxmax(dim='dayofyear').where(mask)
early_clim_min = cml_rain_anom_early.idxmin(dim='dayofyear').where(mask)
late_clim_min = cml_rain_anom_late.idxmin(dim='dayofyear').where(mask)

### Determine regions with double peaks and mask

In [ ]:
# def count_peaks_1d(time_series, prominence=4):
#     peaks, _ = find_peaks(time_series, prominence=prominence)
#     return len(peaks)

# n_peaks = xr.apply_ufunc(count_peaks_1d,
#                          cml_rain_anom_late,
#                          input_core_dims=[["dayofyear"]],
#                          output_core_dims=[[]],
#                          dask="allowed",
#                          vectorize=True)

# n_peaks.plot.imshow(robust=True, size=5, add_labels=False);
# plt.title('Number of climatolgical cumulative daily\nrainfall anomaly peaks (peaks >= 2.5mm/day)');

In [ ]:
# double_rainfall_peaks = xr.where(n_peaks>1, 0, 1)

### Veg phenology regions mask

In [ ]:
lin_or_circ = 'circular'

p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_{lin_or_circ}_final.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_{lin_or_circ}_final.nc'), crs='EPSG:4326')

nan_mask = ~np.isnan(p_average['POS'])

season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

## Find difference between rainfall climatologies

In [ ]:
diff_clean_max = (late_clim_max - early_clim_max)
diff_clean_min = (late_clim_min - early_clim_min)

In [ ]:
diff_clean_max = (late_clim_max - early_clim_max)
diff_clean_min = (late_clim_min - early_clim_min)

#where jumps are greater than half a year, rectify by add/subtracting 365
diff_clean_max = xr.where(diff_clean_max<-183, diff_clean_max+365, diff_clean_max)
diff_clean_max = xr.where(diff_clean_max>183, diff_clean_max-365, diff_clean_max)

diff_clean_min = xr.where(diff_clean_min<-183, diff_clean_min+365, diff_clean_min)
diff_clean_min = xr.where(diff_clean_min>183, diff_clean_min-365, diff_clean_min)

### Plot EOS differences

In [ ]:
corr_data = [early_clim_max, late_clim_max, (diff_clean_max)] 
products=['EOS 1980-2000', 'EOS 2001-2022','EOS Difference (recent - past)'] 

fig,axes = plt.subplots(1,3, figsize=(14,5), sharey=True, layout='constrained')

for ax, dss, n in zip(axes.ravel(), corr_data, products):
    if n=='EOS Difference (recent - past)':
        cmap='PiYG'
        vmin=-50
        vmax=50
    else:
        cmap='twilight'
        vmin=0
        vmax=365   #.where(high_daily_variance).where(double_rainfall_peaks)
    im = dss.where(seasonality_mask!=1).where(nan_mask).plot(cmap=cmap, ax=ax, vmin=vmin, vmax=vmax, add_colorbar=False)   
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    if n=='EOS Difference (recent - past)':
        cbar = fig.colorbar(im, orientation='horizontal',
                ax=ax,
                cmap=cmap,
                ticks=[-50,-40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                shrink=0.80
               )
        cbar.ax.set_title('<--shifted early - days - shifted later-->', fontsize=10)
        cbar.ax.tick_params(labelsize=10)
    else:
        cbar = fig.colorbar(im, orientation='horizontal',
                        ax=ax,
                        norm = mpl.colors.Normalize(vmin=0, vmax=365),
                        ticks=np.arange(0,365, 50),
                        cmap=cmap,
                        shrink=0.80
                       )
        # cbar.set_ticklabels(["J","F","M","A","M","J","J","A","S","O","N","D"], fontsize=10)
        cbar.ax.set_title('Rainfall EOS dayofyear', fontsize=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_title(f'{n}', fontsize=12)
    ax.set_ylabel('')
    ax.set_xlabel('')

# plt.subplots_adjust(wspace=0.075)
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/rainfall_EOS_shifting_patterns.png',
            bbox_inches='tight', dpi=300);

### Plot SOS differences

In [ ]:
corr_data = [early_clim_min, late_clim_min, (diff_clean_min)] 
products=['SOS 1980-2000', 'SOS 2001-2022','SOS Difference (recent - past)'] 

fig,axes = plt.subplots(1,3, figsize=(14,5), sharey=True, layout='constrained')

for ax, dss, n in zip(axes.ravel(), corr_data, products):
    if n=='SOS Difference (recent - past)':
        cmap='PiYG'
        vmin=-50
        vmax=50
    else:
        cmap='twilight'
        vmin=0
        vmax=365 #.where(nan_mask).where(high_daily_variance).where(double_rainfall_peaks)
    im = dss.where(seasonality_mask!=1).plot(cmap=cmap, ax=ax, vmin=vmin, vmax=vmax, add_colorbar=False) 
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    if n=='SOS Difference (recent - past)':
        cbar = fig.colorbar(im, orientation='horizontal',
                ax=ax,
                cmap=cmap,
                ticks=[-50,-40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                shrink=0.80
               )
        cbar.ax.set_title('<--shifted early - days - shifted later-->', fontsize=10)
        cbar.ax.tick_params(labelsize=10)
    else:
        cbar = fig.colorbar(im, orientation='horizontal',
                        ax=ax,
                        norm = mpl.colors.Normalize(vmin=0, vmax=365),
                        ticks=np.arange(0,365, 50),
                        cmap=cmap,
                        shrink=0.80
                       )
        # cbar.set_ticklabels(["J","F","M","A","M","J","J","A","S","O","N","D"], fontsize=10)
        cbar.ax.set_title('Rainfall SOS dayofyear', fontsize=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_title(f'{n}', fontsize=12)
    ax.set_ylabel('')
    ax.set_xlabel('')

# plt.subplots_adjust(wspace=0.075)
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/rainfall_SOS_shifting_patterns.png',
            bbox_inches='tight', dpi=300);

## Calculate lengths of rainy season

In [ ]:
length_early = early_clim_max - early_clim_min
length_late = late_clim_max - late_clim_min

length_early = xr.where(length_early<0, length_early+365, length_early) #negative means min is higher than max so invert
# length_early = xr.where(length_early>300, length_early-365, length_early) #if the season is really long then...

length_late = xr.where(length_late<0, length_late+365, length_late) #negative means min is higher than max so invert
# length_late = xr.where(length_late>300, length_late-365, length_late) #if the season is really long then...

In [ ]:
fig,ax= plt.subplots(1,3, figsize=(16,4), sharey=True)
early_clim_min.plot(cmap='twilight', ax=ax[0])
early_clim_max.plot(cmap='twilight', ax=ax[1])
length_early.plot(ax=ax[2], robust=True);
ax[0].set_title('SOS')
ax[1].set_title('EOS');

In [ ]:
fig,ax= plt.subplots(1,3, figsize=(16,4), sharey=True)
late_clim_min.plot(cmap='twilight', ax=ax[0])
late_clim_max.plot(cmap='twilight', ax=ax[1])
length_late.plot(ax=ax[2], robust=True);
ax[0].set_title('SOS')
ax[1].set_title('EOS');

### Change in LOS

In [ ]:
length_change = length_late - length_early

In [ ]:
corr_data = [diff_clean_min, diff_clean_max, length_change] 
products=['Rain SOS Difference (recent - past)', 'Rain EOS Difference (recent - past)','Change in rainy season length'] 

fig,axes = plt.subplots(1,3, figsize=(14,5), sharey=True, layout='constrained')

for ax, dss, n in zip(axes.ravel(), corr_data, products):
    if "Difference" in n:
        cmap='PiYG'
        vmin=-50
        vmax=50
    else:
        cmap='PRGn'
        vmin=-60
        vmax=60  #.where(nan_mask).where(high_daily_variance).where(double_rainfall_peaks)
    
    im = dss.where(seasonality_mask!=1).plot(cmap=cmap, ax=ax, vmin=vmin, vmax=vmax, add_colorbar=False) 
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    if "Difference" in n:
        cbar = fig.colorbar(im, orientation='horizontal',
                ax=ax,
                cmap=cmap,
                ticks=[-50,-40, -30, -20, -10, 0, 10, 20, 30, 40, 50],
                shrink=0.80
               )
        cbar.ax.set_title('<--shifted early - days - shifted later-->', fontsize=10)
        cbar.ax.tick_params(labelsize=10)
    else:
        cbar = fig.colorbar(im, orientation='horizontal',
                        ax=ax,
                        norm = mpl.colors.Normalize(vmin=-60, vmax=60),
                        ticks=np.arange(-60,70, 10),
                        cmap=cmap,
                        shrink=0.80
                       )
        cbar.ax.set_title('Change in length of rainy season (days)', fontsize=10)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_title(f'{n}', fontsize=12)
    ax.set_ylabel('')
    ax.set_xlabel('')

# plt.subplots_adjust(wspace=0.075)
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/rainfall_season_length_change.png',
            bbox_inches='tight', dpi=300);

## Interactive plots

In [ ]:
# assign_crs(diff_clean_min, crs='EPSG:4326').odc.explore(
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False,
#             cmap='PiYG',
#             vmin=-60,
#             vmax=60
#            )

In [ ]:
# assign_crs(diff_clean_max, crs='EPSG:4326').odc.explore(
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False,
#             cmap='PiYG',
#             vmin=-60,
#             vmax=60
#            )

In [ ]:
# assign_crs(length_change, crs='EPSG:4326').odc.explore(
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False,
#             cmap='PRGn',
#             vmin=-60,
#             vmax=60
#            )


## Mega rainfall change Plot

In [ ]:
# gdf.explore(column='SUB_NAME_7',
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             control = True,
#             legend=False
#            )

In [ ]:
locs =[
    'Tallering','Limme', 'Laura Lowlands', #West Warrego Carnegie
    'Eastern Goldfield', 'Scenic Rim', #'Marlborough Plains',
    'Fitzgerald','Lowan Mallee', 'Culgoa-Bokhar'#'Southern Wooded Downs'
      ]
# gdf_locs_bounds = gdf[gdf['SUB_NAME_7'].isin(locs)]
names=[]
polys = []
for idx,row in gdf[gdf['SUB_NAME_7'].isin(locs)].iterrows():
    b = row.geometry.bounds
    poly = box(minx=b[0], miny=b[1],maxx=b[2],maxy=b[3])
    polys.append(poly)
    names.append(row.SUB_NAME_7)

gdf_locs_bounds = gpd.GeoDataFrame(names, geometry=polys).rename({0:'SUB_NAME_7'},axis=1)
gdf_locs_bounds.crs = 'EPSG:4326'

In [ ]:
layout = [
    ['A',  'B',    'C'],
    ['D','center', 'E'],
    ['F','G', 'H'],
]

# Create the mosaic plot
fig, axes = plt.subplot_mosaic(layout, figsize=(15, 12), layout='constrained')

subax=[
    axes['A'], axes['B'],axes['C'],axes['D'],
    axes['E'],axes['F'],axes['G'], axes['H']
      ]

#centre image of change in rainy season length .where(high_daily_variance)
im = length_change.where(seasonality_mask!=1).where(nan_mask).plot(
    cmap='PRGn', ax=axes['center'], vmin=-60, vmax=60, add_labels=False, add_colorbar=False) 

#plot boxes on top of centre image
gdf_locs_bounds.plot(ax=axes['center'], color=None, facecolor="none", edgecolor='tab:red',linewidth=1)

#colourbar insides the plot
axins1 = inset_axes(axes['center'],width="60%",height="5%",loc="lower left", borderpad=1.75)
cbar = fig.colorbar(im,cax=axins1,orientation='horizontal')
cbar.ax.set_title('Change in length of rainy season (days)', fontsize=9)

ctx.add_basemap(axes['center'], source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
axes['center'].set_yticklabels([])
axes['center'].set_xticklabels([])
axes['center'].set_title(None)

for ax,k,v in zip(subax, locs, [gdf_locs_bounds[gdf_locs_bounds['SUB_NAME_7']==x] for x in locs]):
    print(k)
    
    minx,miny,maxx,maxy = v.geometry.bounds.values[0]
    lat=slice(maxy,miny) 
    lon=slice(minx,maxx)
    rain_subset = rain.sel(longitude=lon, latitude=lat).compute()
    # rain_subset = rain.odc(longitude=lon, latitude=lat).compute()
    
    clim_mean_month_early = rain_subset.sel(time=slice('1980','2000')).groupby('time.dayofyear').mean().mean(['latitude', 'longitude'])
    clim_mean_early = rain_subset.sel(time=slice('1980','2000')).mean('time').mean(['latitude', 'longitude'])
    cml_rain_anom_early = (clim_mean_month_early - clim_mean_early).cumsum(dim='dayofyear')
    
    clim_mean_month_late = rain_subset.sel(time=slice('2001','2022')).groupby('time.dayofyear').mean().mean(['latitude', 'longitude'])
    clim_mean_late = rain_subset.sel(time=slice('2001','2022')).mean('time').mean(['latitude', 'longitude'])
    cml_rain_anom_late = (clim_mean_month_late - clim_mean_late).cumsum(dim='dayofyear')
    
    mask = ~np.isnan(clim_mean_month_early.mean('dayofyear'))
    ax1=ax.twinx()
    pl1=(clim_mean_month_early - clim_mean_early).plot(
        ax=ax,label='1980-2000 climatological rainfall anomaly', linestyle='--', linewidth=0.75)
    pl2=cml_rain_anom_early.rolling(dayofyear=30, min_periods=1).mean().plot(
        ax=ax1, label='1980-2000 cumulative rainfall anomaly')
    
    pl3=(clim_mean_month_late - clim_mean_late).plot(ax=ax, 
                                        label='2001-2022 climatological rainfall anomaly',linestyle='--', linewidth=0.75)
    pl4=cml_rain_anom_late.rolling(dayofyear=30, min_periods=1).mean().plot(
        ax=ax1, label='2001-2022 cumulative rainfall anomaly')

    eos_doy = cml_rain_anom_early.rolling(dayofyear=30, min_periods=1).mean().idxmax(dim='dayofyear').values.item()
    eos_rain = cml_rain_anom_early.rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=eos_doy).values.item()
    sos_doy = cml_rain_anom_early.rolling(dayofyear=30, min_periods=1).mean().idxmin(dim='dayofyear').values.item()
    sos_rain = cml_rain_anom_early.rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=sos_doy).values.item()
    eos_doy_late = cml_rain_anom_late.rolling(dayofyear=30, min_periods=1).mean().idxmax(dim='dayofyear').values.item()
    eos_rain_late = cml_rain_anom_late.rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=eos_doy_late).values.item()
    sos_doy_late = cml_rain_anom_late.rolling(dayofyear=30, min_periods=1).mean().idxmin(dim='dayofyear').values.item()
    sos_rain_late = cml_rain_anom_late.rolling(dayofyear=30, min_periods=1).mean().sel(dayofyear=sos_doy_late).values.item()
    
    #add sos
    ax1.plot(sos_doy, sos_rain, 'or')
    ax1.plot(sos_doy_late, sos_rain_late, 'or')
    
    #add eos
    ax1.plot(eos_doy, eos_rain, 'bo')
    ax1.plot(eos_doy_late, eos_rain_late, 'bo')
    
    plots=pl1+pl2+pl3+pl4
    labels = [l.get_label() for l in plots]
    # ax.legend(plots, labels, loc='upper right')
    ax.axhline(0, c='grey')
    ax.grid(alpha=0.75)
    ax.set_title(None)
    ax1.set_title(k)
    ax1.set_xlabel(None)
    ax.set_xlabel(None)
    ax1.set_ylabel(None)
    ax.set_ylabel(None)

fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/rainfall_seasonality_headlinefigure.png',
          bbox_inches='tight', dpi=300);

In [ ]:
# get handles and labels for reuse
label_params = (plots,labels)

figl, axl = plt.subplots()
axl.axis(False)
axl.legend(*label_params, loc="best", fontsize=12, markerscale=2, ncol=4)
figl.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/rainfall_seasonality_legend.png',
          bbox_inches='tight', dpi=300);

## Process daily chirps

https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/

In [ ]:
# base = '/g/data/os22/chad_tmp/Aus_phenology/data/chirps_daily/'
# gbox = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/rain_5km_monthly_1981_2022.nc').odc.geobox
# files = [i for i in os.listdir(base) if i.endswith('nc')]
# files.sort()

# ds = xr.open_mfdataset([base+i for i in files])
# ds = assign_crs(ds, crs='epsg:4236')
# ds = ds.sel(latitude=slice(-45,-10), longitude=slice(111,155))
# ds = ds.to_array(name='rain')
# ds = ds.drop_vars('variable').squeeze()
# ds.attrs['nodata'] = np.nan
# ds = ds.chunk(dict(time=365, latitude=700, longitude=880))
# ds = ds.odc.reproject(how=gbox, resampling='bilinear')
# ds = round_coords(ds)
# ds = ds.rename('rain')
# ds.astype('float32').compute().to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/rainfall_CHIRPS_daily_5km_1981_2023.nc')

### Get daily rainfall from ANUClim

In [ ]:
# import os
# import xarray as xr
# from odc.geo.xr import assign_crs
# import numpy as np

# import sys
# sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
# from _utils import start_local_dask, round_coords

# client = start_local_dask(mem_safety_margin='2Gb')
# client

In [ ]:
# base='/g/data/gh70/ANUClimate/v2-0/stable/day/'
# var =  'rain'

# years = [str(i) for i in range(1960,1981)]
# gbox = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/rain_5km_monthly_1981_2022.nc').odc.geobox

# i=0
# pp = []
# for y in years:
#     print(" {:02}/{:02}\r".format(i + 1, len(years)), end="")
#     ds = xr.open_mfdataset([base+var+'/'+y+'/'+i for i in os.listdir(base+var+'/'+y+'/')])
#     # ds = ds.chunk(dict(lat=500, lon=500, time=-1))
#     ds = assign_crs(ds, crs='epsg:4283') #GDA94
#     ds = ds.drop_vars('crs')[var]
#     ds.attrs['nodata'] = np.nan
#     ds = ds.odc.reproject(gbox, resampling='bilinear').compute()
#     ds = round_coords(ds)
#     pp.append(ds)
#     i+=1

# ds = xr.concat(pp, dim='time').sortby('time')
# ds = ds.compute()
# ds = ds.rename(var)

# ds.astype('float32').to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/data/rainfall_ANUClim_daily_5km_1960_1980.nc')